In [1]:
import cv2
import os
import re
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
import csv
import pandas as pd
import keras
from keras import models,Model
from keras.layers import Dense,GaussianNoise, Dropout,Input
from keras.utils import np_utils
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
%matplotlib inline
from skimage.feature import hog
%matplotlib inline 
from sklearn.cluster import KMeans


/home/jeet/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/jeet/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dataSet = []
person = []
horiSet = []
train_set_x = []
count = 0
folder = "/home/jeet/CV Assignment/ML face images train S"
for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
#         print(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (200,200))
        sift = cv2.xfeatures2d.SIFT_create()
        kp, des = sift.detectAndCompute(img,None)
        kmeans = KMeans(n_clusters=1, random_state=0).fit(des)
#         print(kmeans.cluster_centers_.shape)
        train_set_x.append(kmeans.cluster_centers_)
#         person.append(numbers[0])
        img = np.ravel(img)
        if img is not None:
#             for desc in des: 
            numbers = re.findall('\d+',filename)
            numbers = list(map(int,numbers))
            person.append(numbers[0])
            horiSet.append(img.T)
#             dataSet.append(desc)
            count = count + 1

print(np.array(train_set_x).shape)      
dataSet_test = []
person_test = []
horiSet_test = []
test_set_x = []
folder = "/home/jeet/CV Assignment/ML face images test S"
for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (200, 200))
        sift = cv2.xfeatures2d.SIFT_create()
        kp, des = sift.detectAndCompute(img,None)
        #kp will be a list of keypoints and des is a numpy array of shape Number_of_Keypoints×128.
        kmeans = KMeans(n_clusters=1, random_state=0).fit(des)
        test_set_x.append(kmeans.cluster_centers_)
        img = np.ravel(img)
        if img is not None:
#             for desc in des: 
            numbers = re.findall('\d+',filename)
            numbers = list(map(int,numbers))
            person_test.append(numbers[0])
            horiSet_test.append(img.T)
#             dataSet_test.append(desc)
            count = count + 1
print(np.array(test_set_x).shape , np.array(person_test).shape)

(204, 1, 128)
(70, 1, 128) (70,)


In [3]:
train_set_x = np.reshape(train_set_x,(418,128))
test_set_x = np.reshape(test_set_x,(140,128))

ValueError: cannot reshape array of size 26112 into shape (418,128)

In [ ]:
#Getting data in desired format and mapping the lables
dataSet = np.array(train_set_x)
train = pd.DataFrame(dataSet)

person = np.array(person)
train['id'] = person.transpose()

unique_person = train['id']
unique_person = np.array(unique_person)
unique_person = np.unique(unique_person)
key = range(len(unique_person))

roll_dict = dict( zip(unique_person, key))

mapped_roll = np.vectorize(roll_dict.get)(person)
train['id'] = mapped_roll.transpose()

y = train['id'].values.astype('int64')
images = train.drop(['id'], axis=1, inplace=False)
x = (images.values).astype('uint8')

X_train = x
Y_train = y

dataSet_test = np.array(test_set_x)
test = pd.DataFrame(dataSet_test)

person_test = np.array(person_test)
test['id'] = person_test.transpose()

mapped_roll_test = np.vectorize(roll_dict.get)(person_test)
test['id'] = mapped_roll_test.transpose()
print(test['id'])

temp = np_utils.to_categorical(test['id'])
print(temp[2])
print(temp.shape)

temp = np_utils.to_categorical(test['id'])
print(temp[2])
print(temp.shape)

y_test = test['id'].values.astype('int64')
images = test.drop(['id'], axis=1, inplace=False)
x_test = (images.values).astype('uint8')

X_test = x_test
Y_test = y_test

In [ ]:
#Defining and training of the neural neural network
model = models.Sequential()
layers = 2
units = 256

#Input layer
model.add(Dense(units, input_dim=128, activation='relu'))

#Hidden Layer
for i in range(layers):
    model.add(Dense(146, activation='relu'))
    model.add(Dropout(0.1))
    
#Output layer
model.add(Dense(36, activation='softmax'))

#Building model
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

#Training model
model.fit(X_train,Y_train,batch_size=70,validation_split=0.2,epochs=150,verbose=2)

In [ ]:
predictions = model.predict_classes(X_test, verbose=0)

def write_predictions(predictions, fname):
    pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)), "id": predictions}).to_csv(fname, index=False, header=True)

write_predictions(predictions, "out.csv")


In [ ]:
result = pd.read_csv('out.csv')
result = result['id']
result = np.array(result)
error = np.array(Y_test - result)
error = error[error != 0]
error = error.size/Y_test.size

print('Accuracy : ',1-error)

result

In [ ]:
score = model.predict_proba(X_test)

In [ ]:
precision = dict()
recall = dict()
average_precision = dict()
for i in range(36):
    precision[i], recall[i], _ = precision_recall_curve(temp[:, i],
                                                        score[:, i])
    average_precision[i] = average_precision_score(temp[:, i], score[:, i])

# A "micro-average": quantifying score on all classes jointly
precision["micro"], recall["micro"], _ = precision_recall_curve(temp.ravel(),
    score.ravel())
average_precision["micro"] = average_precision_score(temp, score,
                                                     average="micro")
print('Average precision score, micro-averaged over all classes: {0:0.2f}'
      .format(average_precision["micro"]))

fig = plt.figure()
plt.step(recall['micro'], precision['micro'], color='b', alpha=0.9,
         where='post')
#plt.fill_between(recall["micro"], precision["micro"], step='post', alpha=0.2,
#                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title(
    'Average precision score, micro-averaged over all classes: AP={0:0.2f}'
    .format(average_precision["micro"]))
plt.show()
fig.savefig('SIFT-ANN(Original image).png',dpi=100)